# Data Mining Versuch Recommender Systeme

* Prof: Dr. Johannes Maucher
* Project: Janina Mattes
* Datum: 22.05.2020

[Übersicht Ipython Notebooks im Data Mining Praktikum](Data Mining Praktikum.ipynb)


# Einführung
## Lernziele:
In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __Ähnlichkeit:__ Verfahren zur Bestimmung der Ähnlichkeit zwischen Personen (Kunden) und Elementen (Produkten)
* __Empfehlungssysteme__ Collaborative Filtering 
* __Collaborative Filtering:__ Nutzerbezogener Ansatz und elementbasierter Ansatz

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung
### Recommender Systeme
Recommender Systeme werden im E-Commerce eingesetzt um Werbung in Form von kundenspezifischen Empfehlungen zu verteilen. Weitläufig bekannt sind die Amazon-Empfehlungen, die entweder per e-mail geschickt oder nach dem Log-In in der Web-Page angezeigt werden. Diese Empfehlungen werden in Abhängigkeit von den bisher vom jeweiligen Kunden gekauften bzw. bewerteten Produkten erstellt. In diesem Versuch werden die derzeit wohl am weitest verbreiteteten Verfahren für die Erzeugung kundenspezifischer Empfehlungen vorgestellt, darunter das elementweise Collaborative Filtering, welches z.B. auch von Amazon eingesetzt wird.     

Direkt-Marketing Methoden wie die kundenspezifische Erzeugung und Bereitstellung von Werbung erfordern detaillierte Kunden- und Warenkorbanalysen. Kunden mit ähnlichem Kaufverhalten werden in Kundengruppen zusammengefasst. Die Warenkorbanalyse untersucht u.a. welche Waren bevorzugt im Verbund von der gleichen Person gekauft werden. Damit kann ein Händler Werbung in Form von Empfehlungen individuell und gezielt an seine Kunden richten, abhängig davon welcher Kundengruppe er angehört und welche Produkte bevorzugt von dieser Kundengruppe nachgefragt werden. 

Im ersten Teil der Übung werden fiktive Daten in einer überschaubaren Menge verwendet. Es handelt sich hier um Filmbewertungen. Anhand dieses Beispiels sollen die notwendigen Methoden und Abläufe implementiert und getestet werden. Diese werden im zweiten Teil der Übung auf echte Daten angewandt. Hierzu werden über eine Python-API Daten vom Internet-Meta-Radio _last.fm_ integriert. Auf der Basis dieser Daten sollen dann Musikempfehlungen für last.fm User berechnet werden. 

Recommender Systeme lassen sich mit

* Clustering Verfahren
* Suchalgorithmen
* Collaborativen Filtering 
 
realisieren. Am häufigsten wird hierbei das Collaborative Filtering eingesetzt. Für das Collaborative Filtering wird jeder der $M$ User durch einen $N$-dimensionalen Vektor beschrieben, wobei $N$ die Anzahl der Produkte im Angebot des Händlers ist. Jedes Element im Vektor gehört zu einem speziellen Produkt. Das Element hat den Wert 1, wenn der User dieses Produkt bereits gekauft hat, sonst 0 (andere Wertbelegungen sind möglich, z.B. wenn Produktbewertungen vorliegen). Alle $M$ Zeilenvektoren können zur _User/Item_ Matrix zusammengefasst werden (siehe Abbildung).

![Abbildung User Item Matrix](https://www.hdm-stuttgart.de/~maucher/ipnotebooks/DataMining//Bilder/UserItemMatrix.png "User Item Matrix")

In [ ]:
# Only use this cell if you work from Google Colab
from google.colab import drive
drive.mount('/content/drive')

Das traditionelle **userbasierte Collaborative Filtering (UCF)**, benutzt die Ähnlichkeit zwischen Benutzern: Um für User $U_i$ eine Empfehlung zu erzeugen wird zunächst der diesem User ähnlichste Kunde (oder eine Menge vom ähnlichsten Kunden) ermittelt. Dann werden $U_i$ die Produkte (Items) empfohlen, welche der ähnlichste Kunde gekauft hat, $U_i$ selbst jedoch noch nicht. 

Dieser Ansatz skaliert schlecht im Fall sehr großer _User/Item_-Matrizen. Ausserdem ist er für User, welche erst wenige Produkte gekauft haben unzuverlässig. Besser eignet sich in diesen Fällen das **itembasierte Collaborative Filtering (ICF)**. Es wird u.a. von Amazon.com eingesetzt. Diese Variante benutzt die Ähnlichkeit zwischen Produkten (Items). Dabei sind Produkte umso ähnlicher je mehr Kunden diese Produkte gemeinsam gekauft haben. Für die Produkte welche ein Referenzuser $U_i$ bereits gekauft hat, werden die ähnlichsten Produkte ermittelt. Diese ähnlichsten Produkte werden $U_i$ empfohlen, wenn er sie nicht schon selbst gekauft hat.

Im folgenden Abschnitt werden einige gebräuchliche Metriken für die Berechnung der Ähnlichkeit zwischen Benutzern oder Artikeln vorgestellt. Für Collaboratives Filtering wird sehr häufig das Cosinus - Ähnlichkeitsmaß eingesetzt.

### Gebräuchliche Ähnlichkeitsmaße

Die __euklidische Distanz__ $d_E(\underline{a},\underline{b})$ zwischen zwei n-dimensionalen Vektoren $\underline{a}=(a_1,\ldots,a_n)$ und $\underline{b}=(b_1,\ldots,b_n)$ berechnet sich zu
	$$
	d_E(\underline{a},\underline{b})=\sqrt{\sum_{i=1}^n (a_i-b_i)^2}
	$$
Zwei Vektoren können als umso ähnlicher erachtet werden, je kleiner deren euklidische Distanz ist. 
Ein auf der euklidischen Metrik basierendes Ähnlichkeitsmaß zwischen zwei Vektoren $\underline{a}$ und $\underline{b}$ kann durch 
$$
s_E(\underline{a},\underline{b})=\frac{1}{1+d_E(\underline{a},\underline{b})}
$$
angegeben werden.


__Pearson Korrelation__
Die Ähnlichkeit zwischen zwei Vektoren kann auch durch den Pearson-Korrelationskoeffizient $\rho_{\underline{a},\underline{b}}$ ausgedrückt werden. Er berechnet sich zu
$$
\rho_{\underline{a},\underline{b}}= \frac{1}{N}\cdot \sum\limits_{i=1}^{N}\frac{(a_i-\overline{a})}{\sigma_a} \frac{(b_i-\overline{b})}{\sigma_b}
$$
Dabei bezeichnet $N$ die Länge der Vektoren, $\overline{a}$ den Mittelwert und $\sigma_a$ die Standardabweichung des Vektors $\underline{a}$. 

Der Pearson-Korrelationskoeffizient misst die lineare Abhängigkeit zwischen zwei Vektoren. Der maximale Wert von $+1$ wird erreicht, wenn die durch die beiden Vektoren definierten N Punkte im 2-dimensionalen Raum auf einer ansteigenden Geraden liegen. Der Minimalwert von $-1$ wird erreicht, wenn die Punkte auf einer abfallenden Geraden liegen. Der Betrag des Koeffizienten ist umso kleiner, je stärker die Punkte von einer fiktiven Geraden (kann durch lineare Regression berechnet werden) abweichen. Der Koeffizient ist $0$ wenn keine lineare Abhängigkeit zwischen den Vektoren besteht.


__Cosinus Ähnlichkeitsmaß__
Die Ähnlichkeit zwischen zwei Vektoren kann auch durch den Cosinus $\cos(\underline{a},\underline{b})$ ausgedrückt werden. Er berechnet sich zu
$$
\cos(\underline{a},\underline{b})= \frac{\underline{a} \cdot \underline{b}}{\left\|\underline{a}\right\|\cdot \left\|\underline{b}\right\|}
$$
wobei im Zähler das Skalarprodukt der beiden Vektoren steht und mit $\left\|\underline{x}\right\|$ der Betrag des Vektors $\underline{x}$ bezeichnet wird.

Falls die Vektoren $\underline{a}$ und $\underline{b}$ mittelwertfrei sind, ist der Cosinus-Ähnlichkeitswert gleich dem Pearson-Korrelationswert. In der Dokument- und Textanalyse wird vornehmlich das Cosinus-Ähnlichkeitsmaß verwendet. 


__Russel Rao Ähnlichkeitsmaß__
Die Russel Rao-Ähnlichkeit zwischen zwei binären Vektoren $\underline{a}$ und $\underline{b}$ mißt das Verhältnis zwischen der Anzahl $\alpha$ der Stellen in denen beide Vektoren den Wert 1 haben und der Länge $n$ der Vektoren. Z.B. ist für die Vektoren $\underline{a}=(1,0,1,0,0,1)$ und $\underline{b}=(0,1,1,1,0,1)$ die Russel-Rao-Ähnlichkeit $s_{RR}(\underline{a},\underline{b})=2/6=0.333$.

__Jaccard Ähnlichkeitsmaß__
Die Jaccard-Ähnlichkeit zwischen zwei binären Vektoren $\underline{a}$ und $\underline{b}$ mißt das Verhältnis zwischen der Anzahl $\alpha$ der Stellen in denen beide Vektoren den Wert $1$ haben und der Anzahl der Stellen in denen mindestens einer der beiden Vektoren ungleich $0$ ist. Z.B. ist für die Vektoren $\underline{a}=(1,0,1,0,0,1)$ und $\underline{b}=(0,1,1,1,0,1)$ die Jaccard-Ähnlichkeit $s_{J}(\underline{a},\underline{b})=2/5=0.4$. Die Jaccard Metrik wird in diesem Versuch für die Bestimmung der Ähnlichkeit von _last.fm_-Usern eingesetzt.


## Vor dem Versuch zu klärende Fragen
Eine Untermenge der im Folgenden aufgeführten Fragen wird zu Beginn des Versuchs im Rahmen eines Gruppenkolloqs abgefragt. Auf jede Frage sollte von mindestens einem Gruppenmitglied eine Antwort geliefert werden und jedes Gruppenmitglied muss mindestens eine der gestellten Fragen beantworten können.

**Aufgaben:**

* Beschreiben Sie das Prinzip des userbasierten Collaborativen Filtering (UCF).

Das Prinzip des _userbasierten Collaborativen Filtering (UCF)_ betrachtet die Ähnlichkeit zwischen zwei Nutzern/Kunden mittels verschiedener Ähnlichkeitsmaßen zum Beispiel der euklidische Distanz oder Pearson Korrelation. Dies bedeutet, um einem Kunden $U_i$ eine Produktempfehlung zu machen wird zuerst eine Menge, diesem Kunden $U_i$ ähnlichen, anderen Kunden ermittelt. Auf Basis dieser kann dem Kunden $U_i$ anschließend eine Produktempfehlung gemacht werden für Produkte, die dieser selbst noch nicht gekauft hat, welche jedoch bereits durch den am ähnlichsten Kunden gekauft wurden. 

* Welche Nachteile hat das UCF?

Eine Schwierigkeit des _UCF_ Verfahrens ist, dass dies im Falle sehr großer _User/Item_-Matrizen nur schlecht skaliert. Darüberhinaus ist die Anwendung dieses Prinzips nicht zuverlässig bei Nutzern, welche erst wenige Produkte gekauft haben. 

In der Praxis übertrifft ein Item-Item-Modell in vielen Anwendungsfällen ein Benutzer-Benutzer-Modell, da man sagen kann, dass Produkte (Items) "einfacher" sind als Benutzer. Das bedeutet, dass Produkte (Items) zu einer kleinen Gruppe von Genres gehören, während die Benutzer zum Beispiel sehr unterschiedliche Geschmäcker haben. Außerdem ist die Ähnlichkeit von Items aussagekräftiger als die Ähnlichkeit von Benutzern.

* Worin besteht der Unterschied zwischen UCF und itembasierten Collaborativen Filtering (ICF)?

Das _itembasierte Collaborative Filtering (ICF)_ basiert im Gegensatz zum _UCF_ Prinzip auf der Ähnlichkeit zwischen Produkten (Items). Je mehr Kunden bestimmte Produkte gemeinsam gekauft haben, desto ähnlicher sind diese. Auf dieser Basis werden die ähnlichsten Produkte ermittelt. 

* Gegeben seien die Vektoren 

    \begin{eqnarray*}
    \underline{a} & = & [1,2,3,4,5,6] \\
    \underline{b} & = & [3,3,5,6,7,8] \\
    \end{eqnarray*}
    
    Schreiben Sie eine Python Funktion, die den Mittelwert derartiger Vektoren berechnet. Schreiben Sie eine weitere Funktion, die die Varianz berechnet

#### Mittelwert

In [3]:
import numpy
import pprint as pp

In [4]:
# Define the vectors

a=[1,2,3,4,5,6]
b=[3,3,5,6,7,8]

In [5]:
# 1. Aufgabe Mean Value

def mean(vector):
    sum = 0
    for i in vector:
        sum += i
    return sum/len(vector)

print("Mean of vector a:{}".format(mean(a)))
print("Mean of vector b:{}".format(mean(b)))

Mean of vector a:3.5
Mean of vector b:5.333333333333333


#### Varianz

In [6]:
# 2. Aufgabe Varianz and Deviation

def variance(vector, mean):
    sum = 0
    for i in vector:
        sum += abs(mean - i) ** 2
    return sum/len(vector)

print("Variance of vector a:{}".format(variance(a, mean(a))))
print("Variance of vector b:{}".format(variance(b, mean(b))))

Variance of vector a:2.9166666666666665
Variance of vector b:3.5555555555555554


* Wie groß ist die

    - Euklidische Ähnlichkeit
    - Pearson Ähnlichkeit
    - Cosinus Ähnlichkeit
    
    zwischen den Vektoren $\underline{a}$ und $\underline{b}$? 

In [7]:
# 3. Aufgabe 
# Euklidische, Pearson und Cosinus Ähnlichkeit

import math
import scipy
from scipy import spatial, stats # sonst evtl OS Probleme

- Euklidische Ähnlichkeit

Zwei Vektoren können als umso ähnlicher erachtet werden, je kleiner deren euklidische Distanz ist.

In [8]:
print("Euklidean Distance: {}".format(scipy.spatial.distance.euclidean(a,b)))

Euklidean Distance: 4.58257569495584


- Pearson Ähnlichkeit

Der Pearson-Korrelationskoeffizient misst die lineare Abhängigkeit zwischen zwei Vektoren. Der maximale Wert ist +1 und der Minimalwert -1. Der Betrag des Koeffizienten ist umso kleiner, je stärker die Punkte von einer fiktiven Geraden (kann durch lineare Regression berechnet werden) abweichen. Der Koeffizient ist  0  wenn keine lineare Abhängigkeit zwischen den Vektoren besteht.

In [9]:
print("Pearson Distance/Correlation: {}".format(scipy.stats.pearsonr(a,b)))


Pearson Distance/Correlation: (0.9833434220628547, 0.0004138517691427288)


- Cosinus Ähnlichkeit

Kosinus-Ähnlichkeit ist ein Maß für die Ähnlichkeit zweier Vektoren. Die Kosinus-Ähnlichkeit liegt zwischen −1 (genau entgegengerichtet) und 1 (genau gleichgerichtet). Ein Wert von 0 bedeutet üblicherweise Unabhängigkeit (Orthogonalität). Zwischenwerte zeigen Ähnlichkeit oder Unähnlichkeit an.
In der Dokument- und Textanalyse wird vornehmlich das Cosinus-Ähnlichkeitsmaß verwendet.


Die Berechnung der Kosinusdistanz mit normalisierten Werten ( _zentrierte Kosinusähnlichkeit_ ) erfasst die Intuition besser, da fehlende Bewertungen als "Durchschnitt" behandelt werden und sie kann sowohl mit "harten Bewertern" als auch mit "leichten Bewertern" einfach umgehen. Auf diese Weise zeigt ein Wert nahe Null eine höhere Ähnlichkeit.
Dieses Ähnlichkeitsmaß ist auch bekannt als **Pearson-Korrelation** ( _zentrierte Cosinus-Ähnlichkeit_ ). 

In [11]:
print("Cosinus Similarity: {}".format(1-scipy.spatial.distance.cosine(a,b)))

Cosinus Similarity: 0.9910600847451639


* In welchen Fällen sind Cosinus- und Pearsonähnlichkeit der euklidischen Ähnlichkeit vorzuziehen?

Die euklidische Ähnlichkeit nutzt absolute Werte zur Berechnung der Ähnlichkeit. Das heißt, wenn Werte skaliert werden ändert sich auch die euklidische Ähnlichkeit.

Bei Cosinus- und Pearsonähnlichkeit ändern sich die Werte beim skalieren nicht. Man nutzt diese Ähnlichkeiten wenn man Korrelationen unabhängig von der absoluten Distanz von Werten bestimmen will.

Die Kosinusähnlichkeit betrachtet den Winkel zwischen zwei Vektoren, während die euklidische Ähnlichkeit den Abstand zwischen zwei Punkten misst. Falls die Vektoren _a_, _b_ keinen Mittelwert haben, d.h. mittelwertfrei sind, so ist der Kosinus-Ähnlichkeitswert gleich dem Pearson-Korrelationswert. Die Kosinusähnlichkeit wird im Allgemeinen als Metrik zur Entfernungsmessung verwendet, wenn die Größe der Vektoren keine Rolle spielt. 
Dies geschieht z.B. bei der Arbeit mit Textdaten, die durch Wortzahlen dargestellt werden. 


In [12]:
from IPython.display import Latex
from IPython.display import Image

# Versuchsdurchführung
## Teil 1: Fiktive Filmbewertung
### Daten
Folgende Tabelle enthält die Filmbewertungen von 7 Personen.
from IPython.display import Latex
In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

![Abbildung Bewertung Fiktive Kunden](https://www.hdm-stuttgart.de/~maucher/ipnotebooks/DataMining/Bilder/recommenderFilmRecommendations.PNG)

Die Tabelle ist als Python dictionary _critics_ implementiert. Die Keys des Python-Dictionary definieren die Namen von Personen (Zeilen in der Matrix), die Filme bewertet haben. Die Values sind selbst wieder Dictionarys, welche als Keys die Filmnamen (Spalten in der Matrix) und als Values die jeweilige Filmbewertung (Matrixelment) enthalten.

In [13]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}
}

### Ähnlichkeiten berechnen

Für die Bestimmung der Ähnlichkeit zwischen Personen und Produkten werden in diesem Versuch ein auf der euklidischen Distanz basierendes Ähnlichkeitsmaß und die Pearson-Korrelation verwendet. Beide Ähnlichkeitsmaße sind in den unten definierten Funktionen implementiert. Alle drei hier implementierten Funktionen zur Berechnung der Ähnlichkeit erhalten als Übergabeparameter das oben definierte Dictionary, das die Filmbewertungen enthält und die Namen der zwei Personen, die verglichen werden sollen. 

Zu beachten ist, dass in beiden Funktionen für die Berechnung der Ähnlichkeit zwischen zwei Personen nur die Produkte berücksichtigt werden, welche von beiden Personen schon bewertet wurden. Es handelt sich hier also um modifizierte Ähnlichkeitsfunktionen. 

__Aufgabe:__
Fragen Sie von diesem Dictionary _Toby's_ Bewertung des Films _Snakes on a Plane_ ab und geben Sie diesen Wert aus: 

In [14]:
# Gebe die Bewertung vom Film "Snakes on a Plane" von Toby aus
print(critics.get("Toby").get("Snakes on a Plane"))

4.5


In [15]:
import numpy as np
import scipy.spatial.distance as sci


def sim_euclid(prefs, person1, person2, normed=True):
  ''' Returns a euclidean-distance-based similarity score for 
  person1 and person2. In the distance calculation the sum is computed 
  only over those items, which are nonzero for both instances, i.e. only
  films which are ranked by both persons are regarded.
  If the parameter normed is True, then the euclidean distance is divided by
  the number of non-zero elements integrated in the distance calculation. Thus
  the effect of larger distances in the case of an increasing number of commonly ranked
  items is avoided.
  '''
  # Get the list of shared_items
  si={}
  for item in prefs[person1]:
    if item in prefs[person2]: si[item]=1
  # len(si) counts the number of common ratings
  # if they have no ratings in common, return 0
  if len(si)==0: return 0

  # Add up the squares of all the differences
  sum_of_squares=np.sqrt(sum([pow(prefs[person1][item]-prefs[person2][item],2) 
                     for item in prefs[person1] if item in prefs[person2]]))
  if normed:
     sum_of_squares= 1.0/len(si)*sum_of_squares
  return 1/(1+sum_of_squares)


def sim_pearson(prefs,p1,p2):
  '''
  Returns the Pearson correlation coefficient for p1 and p2
  '''
    
  # Get the list of commonly rated items
  si={}
  for item in prefs[p1]: 
    if item in prefs[p2]: si[item]=1

  # if they are no ratings in common, return 0
  if len(si)==0: return 0

  # Sum calculations
  n=len(si)
  
  # Calculate means of person 1 and 2
  mp1=np.mean([prefs[p1][it] for it in si])
  mp2=np.mean([prefs[p2][it] for it in si])
  
  # Calculate standard deviation of person 1 and 2
  sp1=np.std([prefs[p1][it] for it in si])
  sp2=np.std([prefs[p2][it] for it in si])
  
  # If all elements in one sample are identical, the standard deviation is 0. 
  # In this case there is no linear correlation between the samples
  if sp1==0 or sp2==0:
      return 0
  r=1/(n*sp1*sp2)*sum([(prefs[p1][it]-mp1)*(prefs[p2][it]-mp2) for it in si])
  return r


def sim_RusselRao(prefs,person1,person2,normed=True):
  ''' Returns RusselRao similaritiy between 2 users. The RusselRao similarity just counts the number
  of common non-zero components of the two vectors and divides this number by N, where N is the length
  of the vectors. If normed=False, the division by N is omitted.
  '''
  # Get the list of shared_items
  si={}
  commons=0
  for item in prefs[person1]: 
    if prefs[person1][item]==1 and prefs[person2][item]==1:   
        commons+=1
  #print commons
  if not normed:
      return commons
  else:
      return commons*1.0/len(prefs[person1]) 

**Aufgabe:**
1. Geben Sie die euklidische Ähnlichkeit und die Pearson Ähnlichkeit zwischen den Personen _Toby_ und _Lisa Rose_ aus.
2. Diskutieren Sie die unterschiedlichen Ähnlichkeitswert.

In [16]:
# 1. Aufgabe

print('Euklidische Aehnlichkeit: ', sim_euclid(critics,'Lisa Rose','Toby'))
print('Pearson Aehnlichkeit: ', sim_pearson(critics,'Lisa Rose','Toby'))

Euklidische Aehnlichkeit:  0.615911621788925
Pearson Aehnlichkeit:  0.9912407071619302


Die Ähnlichkeitswerte liefern für die User Lisa Rose und Toby ein sehr unterschiedliches Ergebnis. Lisa Rose hat insgesamt 6 Filme bewertet, während Toby nur 3 bewertet hat. Im Falle der euklidischen Ähnlichkeit sind nur die  absoluten Werte in die Berechnung eingeflossen, bei denen von beiden Personen Bewertungen vorliegen. Es ergibt sich daher ein Wert von 0.61.
Die Pearson Ähnlichkeit mit 0.99 ist hier sehr hoch, da beide Personen die Filme ähnlich bewertet haben. Zum Beispiel ist der Film "You, me and dupree" im Vergleich zu den anderen Filmen von beiden am schlechtesten bewertet worden.

__Aufgabe:__
1. Schreiben Sie eine Funktion _topMatches(prefs,person,similarity)_, welche für eine beliebige in _critics_ enthaltene Person die Ähnlichkeitswerte zu allen anderen Personen berechnet und in einer geordneten Liste zurück gibt. Der Funktion soll als Übergabeparameter auch die anzuwendende Ähnlichkeitsfunktion (*sim_euclid* oder *sim_pearson*) übergeben werden können. Berechnen Sie mit dieser Funktion für jede Person die *top matches*, zunächst unter Verwendung der euklidischen- dann unter Verwendung der Pearson-Ähnlichkeit.

In [17]:
# 0. Aufgabe

def topMatches(prefs,person,similarity):

    top_matches = []
    
    if similarity == 'sim_euclid':
        for p in prefs:
            if p != person:
                top_matches.append([sim_euclid(prefs,person,p),p])    
            top_matches.sort(reverse=True)
        return top_matches
  
    elif similarity == 'sim_pearson':
        for p in prefs:
            if p != person:
                top_matches.append([sim_pearson(prefs,person,p),p])   
            top_matches.sort(reverse=True)
        return top_matches

    else:
        return "The given similarity was not correct use 'sim_euclid' or 'sim_pearson'"

In [18]:
# Top matches of all persons

top_matches_euclid = []
top_matches_pearson = []

for name in critics:
    top_matches_euclid.append([name,topMatches(critics, name, 'sim_euclid')])
    top_matches_pearson.append([name,topMatches(critics, name, 'sim_pearson')])

2. Geben Sie mit der implementierten Funktion die *top matches* der Person Toby aus.

In [19]:
# 1. Aufgabe

print(topMatches(critics, 'Toby', 'sim_euclid'))
print(topMatches(critics, 'Toby', 'sim_pearson'))

[[0.6666666666666666, 'Mick LaSalle'], [0.6246387977050463, 'Claudia Puig'], [0.615911621788925, 'Lisa Rose'], [0.5584815598877471, 'Michael Phillips'], [0.5227744249483389, 'Jack Matthews'], [0.5108747069239427, 'Gene Seymour']]
[[0.9912407071619302, 'Lisa Rose'], [0.9244734516419051, 'Mick LaSalle'], [0.8934051474415642, 'Claudia Puig'], [0.6628489803598703, 'Jack Matthews'], [0.3812464258315117, 'Gene Seymour'], [-1.0, 'Michael Phillips']]


In [ ]:
3. Vergleichen Sie die beiden Ähnlichkeitsmaße. Welches Ähnlichkeitsmaß erscheint Ihnen für diesen Anwendungsfall sinnvoller und warum?

Im Fall von Toby ist die pearson Ähnlichkeit die bessere Wahl. Kriterium ist hierbei ob beide Personen die gleichen Filme bereits bewertet haben und wie ähnlich sich die angegebenen Bewertungen sind. Dabei ist Toby am ähnlichsten mit der Person Lisa Rose. Beide haben die selben Filme bewertet und sehr ähnliche Bewertungen abgegeben. Am unähnlichsten ist Toby mit der Person Michael Philipps, was Sinn ergibt, da sie nur für zwei gleiche Filme eine Bewertung abgegeben haben.

Für den Anwendungsfall der Filmempfehlung ist die Pearson Ähnlichkeit sinnvoller, da es Korrelationen aufzeigt, unabhängig von den absoluten Bewertungen. Jeder Mensch bewertet unterschiedlich, manche vergeben z.B. viel häufiger die maximale Punktzahl als andere, die hat bei der Pearson Ähnlichkeit keinen Einfluss.

### Berechnung von Empfehlungen mit User basiertem Collaborative Filtering
Für die Produkte, die von einer Person noch nicht gekauft wurden, sollen Empfehlungen berechnet werden. Die Empfehlungen können ebenfalls Werte zwischen 1 (wird nicht empfohlen) und 5 (wird stark empfohlen) annehmen. Für die Berechnung der Empfehlung werden die Bewertungen des jeweiligen Produkts durch die anderen Personen herangezogen. Dabei werden die Bewertungen der ähnlichen Personen (d.h. hoher Pearson-Korrelationswert) stärker mit einbezogen als die Bewertungen durch Personen mit einem niedrigen Korrelationswert.

__Beispiel:__
Toby hat die Filme _The Night Listener_, _Lady in the Water_ und _Just My Luck_ noch nicht gekauft. Für diese Filme soll für Toby eine Empfehlung berechnet werden.
In der unten aufgeführten Tabelle enthält die zweite Spalte die _Pearson-Ähnlichkeitswerte_ zwischen Toby und den anderen Personen. Die Spalten 3, 5 und 7 enthalten die Bewertungen der Filme _The Night Listener_, _Lady in the Water_ und _Just My Luck_ durch die anderen Personen. Die Spalten 4, 6 und 8 enthalten die jeweilige Filmbewertung gewichtet (mulipliziert) mit den Ähnlichkeitswerten der jeweiligen Person. Es fällt auf, dass in der Tabelle _Michael_ nicht enthalten ist. Das liegt daran, dass _Michael_ und _Toby_ einen negativen Ähnlichkeitswert aufweisen, d.h. deren Interessen sind gegenläufig. Personen mit negativem Ähnlichkeitswert sollten für Empfehlungen nicht berücksichtigt werden.
Die Zeile _Sum_ enthält die Summe aller gewichteten Bewertungen. Aus diesem Wert allein kann die Empfehlung noch nicht abgeleitet werden, da Filme die nur von wenigen Personen bewertet wurden, eine relativ kleine Summe ergeben. Deshalb sollte _Sum_ noch durch die Anzahl der Bewertungen für diesen Film geteilt werden. Oder besser: Nicht durch die Summe der Bewertungen, sondern durch die Summe der relevanten Ähnlichkeitswerte (_KSum_). Der resultierende Empfehlungswert ist in der letzten Zeile eingetragen.


![Abbildung Calculate Recommendation](https://www.hdm-stuttgart.de/~maucher/ipnotebooks/DataMining/Bilder/recommenderFilmCalculation.PNG)


__Aufgabe:__
Schreiben Sie eine Funktion _getRecommendations(prefs,person,similarity)_, mit der die Empfehlungswerte berechnet werden können und bestimmen Sie die Empfehlungswerte für Toby. Der Funktion wird  

* das Dictionary _critics_ mit den Filmbewertungen, 
* der Name der Person, für welche Empfehlungen berechnet werden sollen
* die Methode für die Berechnung der Ähnlichkeit *sim_euclid* oder *sim_pearson*

übergeben. Die Methode soll eine geordnete Liste zurück geben. Jedes Listenelement enthält an erster Stelle den berechneten Empfehlungswert und an zweiter Stelle den Namen des Films. Die Liste soll nach Empfehlungswerten absteigend geordnet sein.

Testen Sie diese Funktion indem Sie die Empfehlungen für _Toby_ berechnen und mit den Werten in der oben aufgeführten Tabelle vergleichen.

In [20]:
import numpy as np
import scipy.spatial.distance as sci

In [21]:
def topMatches(prefs, person, similarity):
    matches = []
    for p in prefs:
        if p != person:
            matches.append((p, similarity(prefs, person, p)))
            
    matches.sort(key=lambda tup: tup[1], reverse=True)
    return matches

In [22]:
import numpy as np
import scipy.spatial.distance as sci

In [23]:
topMatches(critics, "Toby", sim_pearson)

[('Lisa Rose', 0.9912407071619302),
 ('Mick LaSalle', 0.9244734516419051),
 ('Claudia Puig', 0.8934051474415642),
 ('Jack Matthews', 0.6628489803598703),
 ('Gene Seymour', 0.3812464258315117),
 ('Michael Phillips', -1.0)]

#### Vergleich
Beim Vergleichen der Werte fällt auf, dass die Werte fast übereinstimmen. Wir gehen davon aus, dass die geringe Abweichung auf den gerundeten Werten der Tabelle basieren.

In [24]:
def getRecommendations(prefs, person, similarity):
    recommendations = []
    korrelation = topMatches(prefs,person,similarity)
    #Alle Filme in eine Liste schreiben
    filmList = []
    for i in prefs:
        for f in prefs[i]:
            if f not in filmList:
                filmList.append(f)
    #schon bewertete Filme entfernen
    for k,v in prefs[person].items():
        if v>0:
            filmList.remove(k)
    
    for film in filmList:
        sumRating = 0
        sumKorr = 0
        
        for person in korrelation:
            if person[1] >=0:
                #print(person)
                if film in list(prefs[person[0]]):
                    sumRating += person[1]*prefs[person[0]][film]
                    sumKorr += person[1]
        
        recommendations.append((sumRating/sumKorr,film))
    recommendations.sort(reverse=True)
    return recommendations
getRecommendations(critics,"Toby",sim_pearson)

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641622, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

### Berechnung von Empfehlungen mit Item basiertem Collaborative Filtering
In den vorigen Aufgaben wurden Ähnlichkeiten zwischen Personen bestimmt und für Produktempfehlungen benutzt (User basiertes Collaborative Filtering). Jetzt soll die Ähnlichkeit zwischen Produkten berechnet werden und auf der Basis dieser Produktähnlichkeit Empfehlungen berechnet werden (Item basiertes Collaborative Filtering).

Dabei sollen die bereits implementierten Ähnlichkeitsfunktion *sim_euclid* und *sim_pearson* sowie die Ähnlichkeeits-Sortierfunktion *topMatches* unverändert eingesetzt werden.

__Aufgabe:__

1. Implementieren Sie eine Funktion, welche das Bewertungsdictionary *critics* derart transformiert, dass die Funktionen *sim_euclid*, *sim_pearson* und *topMatches* für das Item-basierte CF unverändert eingesetzt werden können. Die transformierte Matrix soll unter dem Namen *transCritics* abgespeichert werden.
2. Schreiben Sie eine Funktion *calculateSimilarItems*, die aus der transformierten Matrix *transCritics* ein Dictionary berechnet, welches die Ähnlichkeit zwischen allen Filmen beschreibt. Die Keys des Dictionary sind die Filmnamen. Die Values sind geordnete Listen, welche die Funktion *topMatches* zurückgibt, wenn sie für die Filme (nicht für die User) aufgerufen wird. Dieses Dictionary wird an das aufrufende Programm zurück geben. 
3. Schreiben Sie eine Funktion *getRecommendedItems*, welche basierend auf dem im unten aufgeführten Beispiel dargestellten Verfahren unter Vorgabe der Bewertungsmatrix und der zu verwendenden Ähnlichkeitsfunktion Produktempfehlungen berechnet.
4. Testen Sie die Funktion indem Sie die Empfehlungen für Toby berechnen und mit den Werten in der unten aufgeführten Tabelle vergleichen

__Erläuterndes Beispiel:__

_Toby_ hat die Filme *The Night Listener*, *Lady in the Water* und *Just My Luck* noch nicht gekauft. Für diese Filme soll für *Toby* eine Empfehlung berechnet werden. Gekauft und bewertet hat *Toby* die Filme *Snakes on a plane*, *Superman Returns* und *You and me and Dupree*. Diese bereits vorhandenen Filme bilden die erste Spalte der unten dargestellten Matrix. In der zweiten Spalte befinden sich _Toby's_ Bewertungen dieser Filme. Die Spalten 3,5 und 7 enthalten die Ähnlichkeitswerte (mit *calculateSimilarItems* unter Verwendung des normierten euklidischen Ähnlichkeitsmaßes berechnet) zwischen den drei von *Toby* noch nicht gekauften Filmen und den drei von _Toby_ bewerteten Filmen. Diese Ähnlichkeitswerte werden jeweils mit _Toby's_ Bewertungen multipliziert. Das Resultat dieser Multiplikation befindet sich in den Spalten 4,6 und 8. Der finale Empfehlungswert für die von _Toby_ noch nicht gekauften Filme wird berechnet in dem in den Spalten 4,6 und 8 zunächst die Summe über die Werte dieser Spalte in den drei oberen Zeilen berechnet wird und durch die Summe über die Werte der Spalten 3,5 und 7 geteilt wird. Im Fall, dass die *Pearson-Korrelation* zwischen den Filmen als Ähnlichkeitswert herangezogen wird, können negative Ähnlichkeitswerte auftreten. Dann soll in die Berechnung eines Empfehlungswert für Film A nur dann die Bewertung von Film B einfließen, wenn der Korrelationswert zwischen beiden $>0$ ist.  

![Abbildung Calculate Itembased Recommendation](https://www.hdm-stuttgart.de/~maucher/ipnotebooks/DataMining/Bilder/recommenderFilmItemBased.PNG)

In [25]:
import collections

1. Implementieren Sie eine Funktion, welche das Bewertungsdictionary *critics* derart transformiert, dass die Funktionen *sim_euclid*, *sim_pearson* und *topMatches* für das Item-basierte CF unverändert eingesetzt werden können. Die transformierte Matrix soll unter dem Namen *transCritics* abgespeichert werden.

#### Transformation des Dictionary

Da wir in dieser Sektion das Item basierte CF einsetzen, wird das ursprüngliche Dictionary _critics_ transformiert, sodass anstelle des Nutzers als Key ein Produkt/Item verwendet wird.

In [26]:
# 1. Aufgabe

def transform(dictionary):
    transDict = collections.defaultdict(dict)
    for user, itemDict in dictionary.items():
        for i in itemDict:
            transDict[i][user] = itemDict[i] 
    return dict(transDict)

In [40]:
transCritics = transform(critics)

2. Schreiben Sie eine Funktion *calculateSimilarItems*, die aus der transformierten Matrix *transCritics* ein Dictionary berechnet, welches die Ähnlichkeit zwischen allen Filmen beschreibt. Die Keys des Dictionary sind die Filmnamen. Die Values sind geordnete Listen, welche die Funktion *topMatches* zurückgibt, wenn sie für die Filme (nicht für die User) aufgerufen wird. Dieses Dictionary wird an das aufrufende Programm zurück geben. 

In [27]:
# 2. Aufgabe

def calculateSimilarItems(transCritics, similarity):
    similarDict = {}
    for movie in transCritics:
        similarDict[movie] = topMatches(transCritics, movie, similarity)
    return similarDict    

3. Schreiben Sie eine Funktion *getRecommendedItems*, welche basierend auf dem im unten aufgeführten Beispiel dargestellten Verfahren unter Vorgabe der Bewertungsmatrix und der zu verwendenden Ähnlichkeitsfunktion Produktempfehlungen berechnet.

In [28]:
# 3. Aufgabe

def getRecommendedItems(prefs, person, similarity):
    recommendations = []
    filmKorrelation = calculateSimilarItems(transCritics, similarity)
    #Alle Filme in eine Liste schreiben
    filmList = []
    for i in prefs:
        for f in prefs[i]:
            if f not in filmList:
                filmList.append(f)
    #schon bewertete Filme entfernen
    filmList = set(filmList) - set(prefs[person])    
    
    for film in filmList:
        sumKorr = 0
        sumRatingKorr = 0.0
        for ratedFilm in prefs[person]:
            rating = prefs[person][ratedFilm]
            for korrelation in filmKorrelation[ratedFilm]:
                if korrelation[0] == film and korrelation[1] > 0:
                    sumRatingKorr += rating * korrelation[1]
                    sumKorr += korrelation[1]            
           
        if sumKorr == 0:
            recommendations.append((0, film))
        else:    
            recommendations.append((sumRatingKorr/sumKorr, film))
        
    recommendations.sort(reverse=True)    
    return recommendations

4. Testen Sie die Funktion indem Sie die Empfehlungen für Toby berechnen und mit den Werten in der unten aufgeführten Tabelle vergleichen

In [30]:
getRecommendedItems(critics, "Toby", sim_euclid)

[(3.20449096016088, 'The Night Listener'),
 (3.082136961799338, 'Lady in the Water'),
 (3.041861869079099, 'Just My Luck')]

Beim Vergleichen der Werte fällt auf, dass die Werte fast übereinstimmen. Man kann daher davon ausgehen, dass die geringe Abweichung auf den gerundeten Werten der Tabelle basieren.

## last.fm Musikempfehlungen
Kopieren Sie die Datei _pylast.py_ vom _Resources_-Ordner im _DataMining_-Ordner des Skripteservers in das Verzeichnis dieses _IPython Notebooks_. In dieser Datei sind alle Zugriffsfunktionen auf _last.fm_ Dienste implementiert. Die notwendigen Anmelde- und Authentifizierungsdaten für den User _pythonlab_ sind ebenfalls schon in diesem Modul eingetragen.

In [43]:
!pip install pylast
!pip install matplot
!pip install pandas

import pandas as pd
import pylast
import os 

  Using cached matplot-0.1.9-py2.py3-none-any.whl (5.0 kB)
  Using cached pyloco-0.0.139-py2.py3-none-any.whl (60 kB)
Processing c:\users\janina\appdata\local\pip\cache\wheels\9f\d7\59\3026f39bd579bf6b18560faa960fd38957f31fe5ed845433e3\simplewebsocketserver-0.1.1-cp37-none-any.whl
Processing c:\users\janina\appdata\local\pip\cache\wheels\b8\f2\ed\ea55f33eb524ba8afcfd55e93a6facc96ff23d637e4e609f6d\ushlex-0.99.1-cp37-none-any.whl
  Using cached twine-3.1.1-py3-none-any.whl (36 kB)
  Using cached websocket_client-0.57.0-py2.py3-none-any.whl (200 kB)
  Using cached typing-3.7.4.1-py3-none-any.whl (25 kB)
  Using cached readme_renderer-26.0-py2.py3-none-any.whl (15 kB)
  Using cached requests_toolbelt-0.9.1-py2.py3-none-any.whl (54 kB)
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached pywin32_ctypes-0.2.0-py2.py3-none-any.whl (28 kB)


#### Aufgabe:

1. Stellen Sie durch Aufruf der Funktion *network=pylast.get_lastfm_network()* eine Verbindung zu *last.fm* her. Beim Aufruf der Funktion wird die Anmeldung und Authentifizierung durchgeführt. Die Funktion gibt ein Objekt der Klasse *Network* zurück. Über dieses Objekt können Methoden, wie

    * *get_artist("kuenstlerName")* (liefert Objekt der Klasse _Artist_)
    * *get_album("albumName")* (liefert Objekt der Klasse _Album_)
    * *get_track("songName")* (liefert Objekt der Klasse _Track_)
    * *get_user("userName"):* (liefert Objekt der Klass_Tag_)
    * usw.
    
      aufgerufen werden. Die Menge aller verfügbaren Klassen und deren Attribute und Methoden können dem Modul _pylast.py_ entnommen werden.


In [44]:
# API key and secret key - All keys are stored in a private '.env' environment variable file.
API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")
# Authentication order to perform a write operation 
username = os.environ.get("username")
password_hash = pylast.md5(os.environ.get("password_hash"))

In [45]:
# 1. Aufgabe

# Connect to last.fm API
network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET, username=username, password_hash=password_hash)

Über das Pylast Framework wird die Verbindung zur lastFM API hergestellt. Ebenso wird hier auch das Errorhandling durch die pylast library übernommen. Verschiedene angebotene Methoden ermöglichen das Abrufen von Daten der lastFM API endpoints.


2. Rufen Sie über das oben instanziierte _Network_-Objekt die Methode *get_artist("BandIhrerWahl")* auf.

In [46]:
# 2. Aufgabe

# Keyword to search for
band = "Slipknot"
# Retrieve artist object from network
artist = network.get_artist(band)

3. Rufen Sie über das oben instanziierte _Artist_-Objekt die Methode *topfans=get_top_fans(10)* auf. Die Methode gibt eine Liste von _User_-Objekt/Gewichtung-Paaren zurück. Die Gewichtungen von Objekten werden in diesem Versuch nicht benötigt. Legen Sie deshalb mit _group=[a.item for a in topfan]_ eine Liste an, die nur noch die User Objekte enthält. **Wichtige Anmerkung:** Seit August 2015 gibt es Probleme mit der lastFM API Methode *get_top_fans()* (siehe auch: [pylast issues](https://github.com/pylast/pylast/issues/155s)). Falls am Versuchstermin der Fehler noch nicht behoben ist, können Sie den unten stehenden Code benutzen. Darin wird versucht auf die API-Methode zuzugreifen. Falls das nicht möglich ist, wird eine vordefinierte Liste von Usern angewandt. Diese Liste repräsentiert die *Top Fans* der Band *Slipknot* im Frühjahr 2015. 

In [47]:
def createGroup(usernames):
    group=[]
    # Generate group of users
    for user in usernames:
        u = network.get_user(user)
        group.append(u)
    return group

In [48]:
# 3. Aufgabe
# Retrieve data of top fans - hint the top_fans endpoint got removed at the last.fm API
# In this case a dummy list with users from Slipknot in 2015 are used

usernames=['BrunoJoS','DPREBOYE','MPistol40','NemoNightfall','SkyRif','Wags1382','Znapsen','cortapsyco','emill_67','sattuviitana']
# Generate list of real user objects
group = createGroup(usernames)

3. Implementieren Sie eine Funktion *createLastfmUserDict()*. Dieser Funktion soll, die oben angelegte Liste von _User_-Objekten _group_ übergeben werden. Für jeden User in *group* sollen die 20 beliebtesten Bands mit der Methode *topartists=get_top_artists()[0:20]* bestimmt werden. Die Methode gibt eine Liste von *Artist*-Objekt/Gewichtung-Paaren zurück. Die Gewichtungen von Objekten werden in diesem Versuch nicht benötigt. Auf das *i.te* *Artist*-Objekt selbst kann mit *topartists[i].item* zugegriffen werden. Die Menge aller Bands, die auf diese Weise gesammelt werden, wird im folgenden mit _AllBands_ bezeichnet. D.h. in *AllBands* befinden sich alle Bands, die für mindestens einen User in *group* zu den Top-20 gehören. Nun soll ein verschachteltes Dictionary mit Namen *userDict* wie folgt angelegt werden:

    * Die Keys sind die Namen der _User_-Objekte in _group_. Auf den Namen eines Objekts kann mit *get_name()* zugegriffen werden.
    * Die Values sind selbst wieder Dictionaries, deren Keys die Namen der Bands in *AllBands* sind. Achten Sie auch hier darauf, dass Sie nicht das *Artist*-Objekt selbst, sondern dessen Namen als Key verwenden. 
    * Für den User *a* und die Band *b* ist der Value *userDict[a][b]= 1*, falls *b* zu den Top-20 des Users *a* gehört. Andernfalls ist *userDict[a][b]= 0*. 
    
    Das derart angelegte Dictionary soll von der Funktion zurückgegeben werden. 

In [49]:
# 4. Aufgabe

""" 
The function createLastfmUserDict creates a UserDict dictionary with all users
and their top twenty artists with additional values of 0 or 1 to rate if a user has selected the
band before to their favorites or not.
"""
def createLastfmUserDict(usergroup):
    userDict = {}
    allbands = []
    
    for user in usergroup:
        bands = []
        # api call to fetch twenty most pop artists
        topartists = user.get_top_artists()[:20]                
        # filter all unique bands
        for i in range(len(topartists)):
            if topartists[i] in allbands:
                pass
            else:
                allbands.append(topartists[i].item)
                bands.append(topartists[i].item.get_name())
        # store user values in nested dict
        userDict[user.get_name()] = bands
    
    # fill user dictionary
    for user, bands in userDict.items():
        banddict = {}
        # iterate through all bands
        for i in range(len(allbands)):
            # check if bandname fits
            if allbands[i].get_name() in bands:
                banddict[allbands[i].get_name()] = 1
            else:
                banddict[allbands[i].get_name()] = 0                
        userDict[user] = banddict
    return userDict    

5. Wählen Sie jetzt einen beliebigen User aus *group*. Bestimmen Sie zu diesem User die ähnlichsten User in *group* durch Anwendung der im ersten Teilversuch implementierten Funktion *topMatches()*. Der Funktion wird das angelegte *userDict* und der Name des gewählten Users übergeben. Als Ähnlichkeitsmaß soll die euklidische Metrik angewandt werden.

In [53]:
# 5. Aufgabe

# picked the user 'NemoNightfall'
user = 'NemoNightfall'
userDict = createLastfmUserDict(group)
# print(userDict)

# Calculate the most similar users to 'user'
similarityList = topMatches(userDict, user, sim_euclid)
print('Most similar users to {} are: \n{}'.format(user, similarityList))

WSError: Invalid API key - You must be granted a valid key by last.fm

5. Bestimmen Sie dann für den gewählten User Band-Empfehlungen durch Anwendung der im ersten Teilversuch implementierten Funktion *getRecommendations()*. Der Funktion wird das angelegte *userDict* und der Name des gewählten Users übergeben. Als Ähnlichkeitsmaß soll die euklidische Metrik, danach die Russel_Rao Metrik, angewandt werden.     

In [51]:
# 6. Aufgabe

# Song recommendations for the user
songList1 = getRecommendations(userDict, user, sim_euclid)
songList2 = getRecommendations(userDict, user, sim_RusselRao)

pp.pprint('Recommendet songs for {} are: \n{}'.format(user, songList1))
pp.pprint('Recommendet songs for {} are: \n{}'.format(user, songList2))

NameError: name 'userDict' is not defined

6. Diskutieren Sie das Ergebnis

In [54]:
difList = []
for band1 in songList1:
    for band2 in songList2:
        if band1[1] == band2[1]:
            scoreDif = band1[0] - band2[0]
            difList.append((scoreDif, band1[1]))
difList.sort(reverse = True)

difDf = pd.DataFrame(difList)
difDf['colors'] = ['red' if x < 0 else 'green' for x in difDf[0]]
difDf
plt.figure(figsize = (14, 30), dpi = 80)
plt.hlines(y=difDf[1], xmin=0, xmax=difDf[0], color=difDf.colors, linewidth=5)
plt.yticks(fontsize=10)
plt.gca().set(ylabel="$Bands$", xlabel="$Difference$")
plt.title("Difference between between Euclid and Russel Rao", fontdict={'size':20})
plt.grid(linestyle="-",alpha=0.5)
plt.show()

ModuleNotFoundError: No module named 'pandas'

In [52]:
import matplotlib.pyplot as plt
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 8}

plt.rc('font', **font)

In [ ]:
songList1 = sorted(songList1[:10])
songList2 = sorted(songList2[:10])

x_val = [format(song[0]*100, '.2f') for song in songList1]
y_val = [song[1] for song in songList1]
x_val1 = [format(song[0]*100, '.2f') for song in songList2]
y_val1 = [song[1] for song in songList2]

fig, axs = plt.subplots(2, 1, figsize=(20, 6), sharey=True)
axs[0].bar(y_val, x_val)
axs[1].bar(y_val1, x_val1)
axs[0].set_yticks(np.arange(0, 10.0, step=10.0))
axs[1].set_yticks(np.arange(0, 10.0, step=10.0))
fig.suptitle('Band Recommendations')

#### Diskussion der Ergebnisse

Bei der Untersuchung der ersten zehn Band-Empfehlungen mit dem höchsten Wahrscheinlichkeitswert für deren Ähnlichkeit zu von anderen Personen favorisierten Bands, konnte unter der Hinzunahme der _euklidischen Distanz_ , als auch der _Russel Rao_ Metrik ein sehr ähnliches Ergebnis erzielt werden. In beiden Ergebnisslisten finden sich die Bands _Korn_ (euclid/56%, russel_rao/59.9%), _Rage Against the Machine_  (euclid/44%, russel_rao/43.9%), als auch _Rammstein_  (euclid/34%, russel_rao/43.9%) unter den ersten drei Empfehlungen. Abnehmend von hier unterschiedet sich das Ergebnis leicht im berechneten Wert, inhaltlich sind die ersten zehn Bands jedoch namentlich in beiden Listen vertreten. 

Wahrscheinlich ist aufgrund der Empfehlungswerte, dass der beispielhaft gewählte Nutzer "NemoNightfall" eine Tendenz zu alternativen Musikrichtungen, wie Metal, als auch Rock hat. 

#### Zusatzaufgabe LightFM Library

This is an experiment with our testuser 'NemoNightFall' of the Last.fm database. The idea is to fetch his favorite bands and train the LightFM algorithm based on previously rated songs. 


In [ ]:
#!pip install lightfm
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

In [ ]:
data = fetch_movielens(min_rating=5.0)
print(repr(data['train']))
#testuser = network.get_user(user)

In [ ]:
# model training via sgd (stochastic gradient descent), 
# for every epoch the model learns to fit the data more and more closely.
model = LightFM(loss='warp')
%time model.fit(data['train'], epochs=30, num_threads=2)

In [ ]:
# evaluate the model 
print("Train precision: %.2f" % precision_at_k(model, data['train'], k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, data['test'], k=5).mean())

In [ ]:
def sample_recommendation(model, data, user_ids):
    

    n_users, n_items = data['train'].shape

    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        
        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]
        
        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")
        
        for x in top_items[:3]:
            print("        %s" % x)
        
sample_recommendation(model, data, [3, 25, 450])